### Notes and thoughts

#### Current best ideas / runs that work / etc.
- Batch size = ~1/6th of training data size seems to make all the difference
- nfilters is only 4 for a single class (perhaps 8 works better? Takes longer to train. Trade-offs...)
- Perhaps the best method is to train 10 models, one for each class
- Dropout after each maxpooling step in the down-path and after each upconv/conv/conv step in the up-path. I used p=0.2 for each dropout layer, however the code is written such that this can be changed ona  per-layer basis.
- Activation = LeakyReLu
- Batchnorm is *after* the activation
- L2 reg is applied to each convolution with a lambda of 0.00001. Question remains: is this even doing anything?
- lr is 0.001. This is the default for Adam. Would a higher value train faster?
- Currently there are 400 training images
    - Could this be reduced?
    - Currently the data augmentation is applied to the entire data set, then it is randomly split for validation
    - Should the test/val sets be split early and *then* apply data augmentation *seperately*?
    - **NB** be aware that some classes only have a single image. Thus, the training/val sets should each contain examples of this class but it should be heavily augmented so as to be treated as different)

#### Ideas
- Weight map: sum the total area of all classes in y, then calculate each class' proportion of the total and use `1-value` in place of 1 in the binary mask. This will cause low frequency classes to contribute more to the total loss, i.e. penalizing the model when it fails to predict low frequency classes.

#### Data augmentation / image manipulation
- [Histogram Equalization](http://scikit-image.org/docs/dev/auto_examples/color_exposure/plot_equalize.html#sphx-glr-auto-examples-color-exposure-plot-equalize-py) (also see [here](https://www.kaggle.com/gabrielaltay/dstl-satellite-imagery-feature-detection/exploring-color-scaling-images/discussion), [here](http://adilmoujahid.com/posts/2016/06/introduction-deep-learning-python-caffe/) and [here](https://medium.com/@vivek.yadav/improved-performance-of-deep-learning-neural-network-models-on-traffic-sign-classification-using-6355346da2dc#.x9nidcsh6))
- Rotation (with reflection): see data-augmentation.ipynb
- Image normalization: see image-preprocessing-new.ipynb

#### Overfitting solutions
- CNNs are supposed to be more robust to this because of the shared weight matrix of each filter
- **Data augmentation!**
    - Have done random rotations on data increasing total n by 6-fold
    - Not seeing drastic improvements
- L2 regularization (added into the layers via `W_regularizer=l2(l=0.01)` parameter)
    - Not seeing much improvement
    - Currently set to 0.00001
- Move batchnorm to *before* the relu takes place (see [here](http://stackoverflow.com/questions/34716454/where-do-i-call-the-batchnormalization-function-in-keras)
- Add batchnorm to upconv() layer
- Try mode=1 in batchnorm
- Dropout hurts the model in my experience...

##### U-Net architecture

See [here](https://github.com/jocicmarko/ultrasound-nerve-segmentation/blob/master/train.py#L19) for code and [here](https://arxiv.org/pdf/1505.04597.pdf) for the original literature.

In [1]:
%reset
import numpy as np
from keras.models import Model
from keras.layers import Input, merge, Activation, Convolution2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, Cropping2D, BatchNormalization, Dropout
from keras.optimizers import Adadelta, Adam
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from keras import backend as K
from keras.regularizers import l2
from keras.constraints import maxnorm
from keras.layers.advanced_activations import LeakyReLU

K.set_image_dim_ordering('th')  # Theano dimension ordering in this code
# "tf" assumes (rows, cols, channels) while "th" assumes (channels, rows, cols)
# Possibly change this around natively in the data so the backend doesn't have to switch them
# Only necessary if I use TF!

from matplotlib import pyplot as plt

Once deleted, variables cannot be recovered. Proceed (y/[n])? 
Nothing done.


Using Theano backend.
Using gpu device 0: GRID K520 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5105)
/home/ubuntu/anaconda3/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


### Helper functions

In [2]:
def norm_preds(preds):
    for i in range(preds.shape[0]):
        preds[i,0,:,:] = (preds[i,0,:,:].min() - preds[i,0,:,:])/(preds[i,0,:,:].min()-preds[i,0,:,:].max())
    return preds

In [3]:
def plot_all(i,classType):
    fig = plt.figure(figsize=(6,6))

    ax1 = fig.add_subplot(221)
    ax1.imshow(preds[i,0,...],cmap='spectral')

    ax2 = fig.add_subplot(222)
    ax2.imshow(np.rint(preds[i,0,...]),cmap='spectral')

    ax3 = fig.add_subplot(223)
    ax3.imshow(x[i,17,...],cmap='Greys')

    ax4 = fig.add_subplot(224)
    ax4.imshow(y_oneclass[i,classType,:,:],cmap='spectral')
    
    plt.show()
#plot_all(2,2,classType,0.5)
#push('PICTURES!','The plots are ready')

In [4]:
# Load the components of the pushbullet API
from pushbullet import Pushbullet

pb = Pushbullet('o.YFPNNPfGRekivaCGHa4qMSgjZt8zJ6FL')
phone = pb.devices[0]

# Run this cell after anything you want to be notified about!
def push(title='Done!',text='Whatever it was, it\'s done'):
    phone.push_note(title,text)

In [5]:
# Import the training data
def import_data():
    x = np.load('./data/x_augmented.npy','r')
    y = np.load('./data/y_augmented.npy','r')
    
    '''with open('./data/x_resized_array.pickle','rb') as f:
        x = pickle.load(f)
        x = x.astype(np.float32)
        
    with open('./data/y_resized_raster.pickle','rb') as f:
        y = pickle.load(f)
        y = y.astype(np.float32)'''
    
    y_oneclass = y[:,3:4,...]
    
    '''
    Classes:
    0 Buildings - large building, residential, non-residential, fuel storage facility, fortified building
    1 Misc. Manmade structures 
    2 Road 
    3 Track - poor/dirt/cart track, footpath/trail
    4 Trees - woodland, hedgerows, groups of trees, standalone trees
    5 Crops - contour ploughing/cropland, grain (wheat) crops, row (potatoes, turnips) crops
    6 Waterway 
    7 Standing water
    8 Vehicle Large - large vehicle (e.g. lorry, truck,bus), logistics vehicle
    9 Vehicle Small - small vehicle (car, van), motorbike
    '''
    
    return x, y, y_oneclass

In [6]:
run = 81

In [7]:
x, y, y_oneclass = import_data()

In [8]:
x.shape[0]

400

In [9]:
%%time
'''
V2.0 U-Net with batchnorm
'''
def builder(img_rows = x.shape[2],img_cols = x.shape[3],
            nfilters = 32,activation = 'relu',init = 'he_normal',
            lr=1.0,decay=0.0,momentum=0.0, nesterov=False,reg=0.01,p=[0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2]):
    
    def jaccard(y_true, y_pred,smooth=1.):
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = K.sum(y_true_f * y_pred_f)
        return (intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth)
    
    def Conv2DReluBatchNorm(n_filter, w_filter, h_filter, inputs, activation, init='he_uniform',dropout=0.2):
        # Batch norm after activation
        return BatchNormalization(mode=2, axis=1)(LeakyReLU()((Convolution2D(n_filter, w_filter, h_filter, border_mode='same',init=init,W_regularizer=l2(reg),W_constraint = maxnorm(3))(inputs))))
        
        # Batch norm before activation
        #return LeakyReLU()(BatchNormalization(mode=0, axis=1)((Convolution2D(n_filter, w_filter, h_filter, border_mode='same',init=init,W_regularizer=l2(reg),W_constraint = maxnorm(3))(inputs))))

    def up_conv(nfilters,filter_factor,inputs,init=init,activation=activation):
        # No batch norm
        #return LeakyReLU()(Convolution2D(nfilters*filter_factor, 2, 2, border_mode='same',init=init,W_regularizer=l2(reg),W_constraint = maxnorm(3))(UpSampling2D(size=(2, 2))(inputs)))
        
        # Batch norm after activation
        return BatchNormalization(mode=2, axis=1)(LeakyReLU()(Convolution2D(nfilters*filter_factor, 2, 2, border_mode='same',init=init,W_regularizer=l2(reg),W_constraint = maxnorm(3))(UpSampling2D(size=(2, 2))(inputs))))

    inputs = Input((20, img_rows, img_cols))
    padded = ZeroPadding2D(padding=(12,12))(inputs)
    
    conv1 = Conv2DReluBatchNorm(nfilters, 3, 3, padded, activation=activation,init=init)
    conv1 = Conv2DReluBatchNorm(nfilters, 3, 3, conv1, activation=activation,init=init)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    pool1 = Dropout(p=p[0])(pool1)

    conv2 = Conv2DReluBatchNorm(nfilters*2, 3, 3, pool1, activation=activation,init=init)
    conv2 = Conv2DReluBatchNorm(nfilters*2, 3, 3, conv2, activation=activation,init=init)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(p=p[1])(pool2)

    conv3 = Conv2DReluBatchNorm(nfilters*4, 3, 3, pool2, activation=activation,init=init)
    conv3 = Conv2DReluBatchNorm(nfilters*4, 3, 3, conv3, activation=activation,init=init)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    pool3 = Dropout(p=p[2])(pool3)

    conv4 = Conv2DReluBatchNorm(nfilters*8, 3, 3, pool3, activation=activation,init=init)
    conv4 = Conv2DReluBatchNorm(nfilters*8, 3, 3, conv4, activation=activation,init=init)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    pool4 = Dropout(p=p[3])(pool4)

    conv5 = Conv2DReluBatchNorm(nfilters*16, 3, 3, pool4, activation=activation,init=init)
    conv5 = Conv2DReluBatchNorm(nfilters*16, 3, 3, conv5, activation=activation,init=init)
    conv5 = Dropout(p=p[4])(conv5)
        
    up6 = merge([up_conv(nfilters,8,conv5), conv4], mode='concat', concat_axis=1)
    conv6 = Conv2DReluBatchNorm(nfilters*8, 3, 3, up6, activation=activation,init=init)
    conv6 = Conv2DReluBatchNorm(nfilters*8, 3, 3, conv6, activation=activation,init=init)
    conv6 = Dropout(p=p[5])(conv6)

    up7 = merge([up_conv(nfilters,4,conv6), conv3], mode='concat', concat_axis=1)
    conv7 = Conv2DReluBatchNorm(nfilters*4, 3, 3, up7, activation=activation,init=init)
    conv7 = Conv2DReluBatchNorm(nfilters*4, 3, 3, conv7, activation=activation,init=init)
    conv7 = Dropout(p=p[6])(conv7)

    up8 = merge([up_conv(nfilters,2,conv7), conv2], mode='concat', concat_axis=1)
    conv8 = Conv2DReluBatchNorm(nfilters*2, 3, 3, up8, activation=activation,init=init)
    conv8 = Conv2DReluBatchNorm(nfilters*2, 3, 3, conv8, activation=activation,init=init)
    conv8 = Dropout(p=p[7])(conv8)

    up9 = merge([up_conv(nfilters,1,conv8), conv1], mode='concat', concat_axis=1)
    conv9 = Conv2DReluBatchNorm(nfilters, 3, 3, up9, activation=activation,init=init)
    conv9 = Conv2DReluBatchNorm(nfilters, 3, 3, conv9, activation=activation,init=init)
    conv9 = Dropout(p=p[8])(conv9)
    
    conv10 = Conv2DReluBatchNorm(1, 1, 1, conv9, activation='relu',init=init)
    cropped = Cropping2D(cropping=((12,12), (12,12)))(conv10)
    output = Activation(activation='sigmoid')(cropped)
    
    model = Model(input=inputs, output=output)
    
    model.compile(optimizer=Adam(lr=lr,decay=decay), loss='binary_crossentropy', metrics=[jaccard])
    
    return model

model = builder(img_rows=x.shape[2],img_cols=x.shape[3],
            nfilters=4,activation='relu',init='he_normal',
            lr=0.001,decay=0,momentum=0,reg=0.00001,p=[0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2])

#push('The model is compiled','')

CPU times: user 1.77 s, sys: 34.7 ms, total: 1.8 s
Wall time: 1.82 s


In [ ]:
run += 1
def train_and_predict(model,fit=True,use_existing=False):
    print('This is run number: '+ str(run) + '...')
    
    if use_existing:
        model.load_weights('./data/model_weights_class_3_run_'+str(run)+'.hdf5')
        
    if fit:
        
        print('Fitting model...')
        print('-'*30)
        
        quitter = EarlyStopping(monitor='loss', min_delta=0.001, patience=100, verbose=1, mode='auto')
        lrreducer = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=10, verbose=1, mode='auto', epsilon=0.001, cooldown=2, min_lr=0)
        model_checkpoint = ModelCheckpoint('./data/model_weights_class_3_run_'+str(run)+'.hdf5', monitor='loss', save_best_only=True)
        csvlogger = CSVLogger('./data/training_log_run_'+str(run), separator=',', append=False)

        #tensorboard = TensorBoard(log_dir='./logs/'+'run_'+str(run), histogram_freq=0, write_graph=True, write_images=False)
        '''
        screen -S tensorboard
        tensorboard --logdir=logs
        <ctrl + a,d to exit>
        screen -r tensorboard
        '''
        
        model.fit(x, y_oneclass,
                  batch_size=80,
                  nb_epoch=1000,
                  verbose=1,
                  shuffle=True,
                  callbacks=[model_checkpoint,csvlogger],
                  validation_split=0.2,
                  initial_epoch=0)
    
        print('Predicting masks on test data...')
        print('-'*30)
        
    imgs_mask_test = model.predict(x, verbose=1)
    np.save('jaccard_preds_all_data.npy', imgs_mask_test)
    
    return model

model = train_and_predict(model,fit=True,use_existing=False)
push('Training is done',
     'Train loss: %f, train jaccard: %f, val loss %f, val jaccard%f' %(model.history.history['loss'][-1],model.history.history['jaccard'][-1],model.history.history['val_loss'][-1],model.history.history['val_jaccard'][-1]))

This is run number: 82...
Fitting model...
------------------------------
Train on 320 samples, validate on 80 samples
Epoch 1/1000
320/320 [==============================] - 6s - loss: 0.7416 - jaccard: 0.0966 - val_loss: 0.6398 - val_jaccard: 0.0966
Epoch 2/1000
320/320 [==============================] - 6s - loss: 0.6401 - jaccard: 0.0952 - val_loss: 0.5735 - val_jaccard: 0.0963
Epoch 3/1000
320/320 [==============================] - 6s - loss: 0.5894 - jaccard: 0.0946 - val_loss: 0.5413 - val_jaccard: 0.0953
Epoch 4/1000
320/320 [==============================] - 6s - loss: 0.5618 - jaccard: 0.0939 - val_loss: 0.5265 - val_jaccard: 0.0951
Epoch 5/1000
320/320 [==============================] - 6s - loss: 0.5458 - jaccard: 0.0941 - val_loss: 0.5192 - val_jaccard: 0.0954
Epoch 6/1000
320/320 [==============================] - 6s - loss: 0.5363 - jaccard: 0.0936 - val_loss: 0.5148 - val_jaccard: 0.0955
Epoch 7/1000
320/320 [==============================] - 6s - loss: 0.5300 - jaccard

In [ ]:
preds = np.load('jaccard_preds_all_data.npy')

In [ ]:
model.evaluate(x[120:150,...],y_oneclass[120:150,...])

In [ ]:
norm = False
if norm:
    preds = norm_preds(preds)
i = np.random.choice(range(120),1)[0]
print(i)
plot_all(i,0)

In [ ]:
def plot_everything():
    fig = plt.figure(figsize=(12,36))
    #num = np.random.randint(10,25,10)
    num = np.arange(0,10)
    for i in range(1,10):
        ax1 = fig.add_subplot(10,8,8*i-7)
        ax1.imshow(preds[num[i],0,...],cmap='spectral')
        
        ax2 = fig.add_subplot(10,8,8*i-6)
        ax2.imshow(y_oneclass[num[i],0,...],cmap='spectral')
        
        ax3 = fig.add_subplot(10,8,8*i-5)
        ax3.imshow(preds[num[i],1,...],cmap='spectral')
        
        ax4 = fig.add_subplot(10,8,8*i-4)
        ax4.imshow(y_oneclass[num[i],1,...],cmap='spectral')
        
        ax5 = fig.add_subplot(10,8,8*i-3)
        ax5.imshow(preds[num[i],2,...],cmap='spectral')
        
        ax6 = fig.add_subplot(10,8,8*i-2)
        ax6.imshow(y_oneclass[num[i],2,...],cmap='spectral')
        
        ax7 = fig.add_subplot(10,8,8*i-1)
        ax7.imshow(preds[num[i],3,...],cmap='spectral')
        
        ax8 = fig.add_subplot(10,8,8*i)
        ax8.imshow(y_oneclass[num[i],3,...],cmap='spectral')
        
    plt.show()
plot_everything()

In [ ]:
plot_classifier(model)

### Defunct bits and pieces

In [ ]:
def load_cancelled(model):
    model.load_weights('./data/model_weights_class_3_run_66.hdf5')
    imgs_mask_test = model.predict(x, verbose=1)
    return imgs_mask_test

preds = load_cancelled(model)

In [ ]:
def plot_classifier(model):
    fig = plt.figure(figsize=(12, 6))

    ax = fig.add_subplot(111)
    ax.plot(model.history.history['loss'], "-",color='blue',label="Training final loss: "+str(round(model.history.history['loss'][-1],4)))
    ax.plot(model.history.history['jaccard'], "-",color='orange',label="testing final loss: "+str(round(model.history.history['jaccard'][-1],4)))
    #ax.set_xlim([0, epochs])
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Jaccard / Loss')
    plt.legend(loc='best')
    ax.set_title('Metrics vs time (epochs)')
    
    '''ax = fig.add_subplot(122)
    ax.plot(model.history['acc'], "-",color='blue',label="Training final acc: "+str(round(model.history['acc'][-1],4)))
    ax.plot(model.history['val_acc'], "-",color='orange',label="testing final acc: "+str(round(model.history['val_acc'][-1],4)))
    #ax.set_xlim([0, epochs])
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Accuracy')
    plt.legend(loc='best')
    ax.set_title('Accuracy vs time (epochs)')    
    '''
    plt.show()

In [ ]:
# Import the training data
def import_data(load_whitened = False, normalize = True):
    if load_whitened:
        with open('./data/x_whitened_resized_array.pickle','rb') as f:
            x = pickle.load(f)
            x = x.astype('float32')
    else:
        with open('./data/x_resized_array.pickle','rb') as f:
            x = pickle.load(f)
            x = x.astype('float32')
            # Normalize data to max values
            for i in range(x.shape[0]):
                for j in range(x.shape[1]):
                    x[i,j,:,:] *= 1/x[i,j,:,:].max()
                    
    # Normalize values between 1 and 0
    if normalize:
        for i in range(x.shape[0]):
            for j in range(x.shape[1]):
                #x[i,j,:,:] /= x[i,j,:,:].max()
                x[i,j,:,:] = (x[i,j,:,:].min() - x[i,j,:,:])/(x[i,j,:,:].min() - x[i,j,:,:].max())

    with open('./data/y_resized_raster.pickle','rb') as f:
        y = pickle.load(f)
        y = y.astype(np.float32)
        
    y_oneclass = y[:,1:6,...]
    
    '''
    Classes:
    0 Buildings - large building, residential, non-residential, fuel storage facility, fortified building
    1 Misc. Manmade structures 
    2 Road 
    3 Track - poor/dirt/cart track, footpath/trail
    4 Trees - woodland, hedgerows, groups of trees, standalone trees
    5 Crops - contour ploughing/cropland, grain (wheat) crops, row (potatoes, turnips) crops
    6 Waterway 
    7 Standing water
    8 Vehicle Large - large vehicle (e.g. lorry, truck,bus), logistics vehicle
    9 Vehicle Small - small vehicle (car, van), motorbike
    '''
    
    return x, y, y_oneclass
    
    # Just use a single class: roads
    #y = y[:,4,:,:]
    #y = y[:,np.newaxis,:,:]

    # y = y.reshape(y.shape[0],-1)
    # x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25, random_state=42)

In [ ]:
def np_jaccard(y_true,y_pred,smooth=1.):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true * np.rint(y_pred))
    return (intersection+smooth) / (np.sum(y_true) + np.sum(y_pred) - intersection+smooth)